In [2]:
import openai 
import os
import inspect, json
from inspect import Parameter
from pydantic import create_model
from openai import ChatCompletion, Completion

from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()

    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [4]:
def askgpt(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return ChatCompletion.create(model=model, messages=msgs, **kwargs)

In [5]:
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b

In [6]:
schema(sums)

/var/folders/xj/ht87czwd6h74xh7b8zjync1w0000gn/T/ipykernel_60551/3125953927.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


{'name': 'sums',
 'description': 'Adds a + b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
  'required': ['a'],
  'title': 'Input for `sums`',
  'type': 'object'}}

In [8]:
askgpt("What is 1 + 1?")

<OpenAIObject chat.completion id=chatcmpl-8HJz6DPzANGRZd0cl1KWDUjell6BJ at 0x10a06a9f0> JSON: {
  "id": "chatcmpl-8HJz6DPzANGRZd0cl1KWDUjell6BJ",
  "object": "chat.completion",
  "created": 1699139088,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The sum of 1 + 1 is 2."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 15,
    "completion_tokens": 12,
    "total_tokens": 27
  }
}